In [16]:
import re
import datetime
import json
import time

In [17]:
import import_ipynb
from MySQL_Load_Data import *
from MongoDB_Load_Data import *

<a id="cell1"></a>
# All functions to be run from below at start of the program

### ----------------------------------------------------------------------------------------------------------------------------------
### Load User_cache

In [18]:
def insert_user_cache(user_cache,query_key,result):
    pinned = 'Y'
    current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")    
    temp= {
        "pinned":pinned,
        "lastused" : current_timestamp,
        "userids" : result
    }
    user_cache[query_key]=temp
    
    return user_cache

### Load Tweet_cache

In [19]:
def insert_tweet_cache(tweet_cache,query_key,result):
    pinned = 'Y'
    current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")    
    temp= {
        "pinned":pinned,
        "lastused" : current_timestamp,
        "tweets" : result
    }
    tweet_cache[query_key]=temp
    
    return tweet_cache

### Load Hashtag_cache

In [20]:
def insert_hashtag_cache(hashtag_cache,query_key,result):
    pinned = 'Y'
    current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")    
    temp= {
        "pinned":pinned,
        "lastused" : current_timestamp,
        "hashtag" : result
    }
    hashtag_cache[query_key]=temp
    
    return hashtag_cache

## -------------------------------------------------------------------------------------------------------------------------------
### Query User Cache

In [21]:
def query_user_cache(user_cache,query_key,query_count):
    
    regex_pattern = r'^(.*?)\s*LIMIT\s*\d+\s*$'    
    match = re.search(regex_pattern, query_key, re.IGNORECASE)    
    if match:
        extracted_match = match.group(1).strip()
    
    match_found = ''
    for key in user_cache.keys():
        if extracted_match in key:
            match_found = 'Y'
            key_query_count = key[key.find("LIMIT")+5:]
            break

    if match_found != 'Y':
        return None
            
    if int(key_query_count) < int(query_count):
        return key
    elif int(key_query_count) >= int(query_count):
        current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")  
        user_cache[key]["lastused"] = current_timestamp
        return user_cache[key]["userids"]
        
    return True

### Query Tweet Cache

In [22]:
def query_tweet_cache(tweet_cache,query_key,query_count):

    regex_pattern = r'^(.*?)\s*LIMIT\s*\d+\s*$'    
    match = re.search(regex_pattern, query_key, re.IGNORECASE)
    if match:
        extracted_match = match.group(1).strip()
    
    match_found = ''
    for key in tweet_cache.keys():
        if extracted_match in key:
            match_found = 'Y'
            key_query_count = key[key.find("LIMIT")+5:]
            break

    if match_found != 'Y':
        return None
            
    if int(key_query_count) < int(query_count):
        return key
    elif int(key_query_count) >= int(query_count):
        current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")  
        tweet_cache[key]["lastused"] = current_timestamp
        return tweet_cache[key]["tweets"]
    
    return True

### Query Hashtag Cache

In [23]:
def query_hashtag_cache(hashtag_cache,query_key,query_count):
    
    regex_pattern = r'^(.*?)\s*LIMIT\s*\d+\s*$'    
    match = re.search(regex_pattern, query_key, re.IGNORECASE)
    if match:
        extracted_match = match.group(1).strip()
    
    match_found = ''
    for key in hashtag_cache.keys():
        if extracted_match in key:
            match_found = 'Y'
            key_query_count = key[key.find("LIMIT")+5:]
            break
            
    if match_found != 'Y':
        return None
            
    if int(key_query_count) < int(query_count):
        return key
    elif int(key_query_count) >= int(query_count):
        current_timestamp = datetime.datetime.now().strftime("%Y-%m-%d:%H:%M:%S.%f")  
        hashtag_cache[key]["lastused"] = current_timestamp
        return hashtag_cache[key]["hashtag"]
        
    return True

### ----------------------------------------------------------------------------------------------------------------------------------
## Save Cache to file

In [24]:
def save_cache(dictionary, filename):
    with open(filename, 'w') as file:
        json.dump(dictionary,file)

## ----------------------------------------------------------------------------------------------------------------------------------
## Reload Cache to dictionary

In [25]:
def load_cache(filename):
    try:
        loaded_dict = {}
        with open(filename, 'r') as file:
            loaded_dict = json.load(file)
#             print(loaded_dict)
            return loaded_dict            
    except FileNotFoundError:
        return loaded_dict  

## -------------------------------------------------------------------------------------------------------------------------------
## Purge Cache entry

In [26]:
def purge_cache(dictionary):
    for key in dictionary:
#         print(key)
        if dictionary[key]['pinned'] == ' ':
            dictionary.pop(key)
            return dictionary
    return dictionary
        
######################################################################################################################

def find_oldest_lastused(dictionary):
    oldest_key = None
    oldest_lastused = None
    for key, value in dictionary.items():
        if value["lastused"] is not None:
            if oldest_lastused is None or value["lastused"] < oldest_lastused:
                oldest_key = key
                oldest_lastused = value["lastused"]
    return oldest_key
        

### Search for User 

In [27]:
def user_search(user_cache,string,Limit):
    
    at_begin_time = time.time()
    
    query_key = string + ' LIMIT ' + str(Limit)
    print(query_key)

    result = query_user_cache(user_cache,query_key,Limit)
    # print(result)
    if result == None:
        print("Going to tables")
        result = query_user(string,Limit,"users") 
        
        if len(user_cache) > 14 : 
            purge_key = find_oldest_lastused(user_cache)
            print("purge_key : ", purge_key)
            user_cache[purge_key]['pinned'] = ' '
            user_cache = purge_cache(user_cache)         
        
        if len(result) > 0:
            user_cache = insert_user_cache(user_cache,query_key,result)
        else:
#             print("No results found")
            result = "No results found"
    elif isinstance(result, list):
        print("found in Cache")
    else:
        print("purge")
        user_cache[result]['pinned'] = ' '
        user_cache = purge_cache(user_cache)
        result = query_user(string,Limita,"users") 
        if len(result) > 0:
            user_cache = insert_user_cache(user_cache,query_key,result)
        else:
#             print("No results found")
            result = "No results found in purge"
    
    at_end_time = time.time_ns()
    elapsed_time = at_end_time - at_begin_time
    print("Time taken:", elapsed_time, "seconds")
    
    return result
        

### Search for Text

In [32]:
def string_search(tweet_cache,start_time, end_time, string, Limit):
    
    at_begin_time = time.time_ns()
    
    query = start_time + end_time + string + 'LIMIT' + str(Limit)
    print(query)

    result = query_tweet_cache(tweet_cache,query,Limit)
    # print(result)
    if result == None:
        print("Going to tables")
        
        result = find_tweet_in_timerange(start_time, end_time, Limit,string) 
        print(result)
        if len(tweet_cache) > 14 : 
            purge_key = find_oldest_lastused(tweet_cache)
            tweet_cache[purge_key]['pinned'] = ' '
            tweet_cache = purge_cache(tweet_cache)        
        
        if len(result) > 0:
            tweet_cache = insert_tweet_cache(tweet_cache, query, result)
        else:
#             print("No results found")
            result = "No results found"
    elif isinstance(result, list):
        print("found in Cache")
    else:
        print("purge")
        tweet_cache[result]['pinned'] = ' '
        tweet_cache = purge_cache(tweet_cache)
        result = find_tweet_in_timerange(start_time, end_time, Limit,string) 
        if len(result) > 0:
            tweet_cache = insert_tweet_cache(tweet_cache, query, result)
        else:
#             print("No results found")
            result = "No results found in purge"
    
    at_end_time = time.time()
    elapsed_time = at_end_time - at_begin_time
    print("Time taken:", elapsed_time, "seconds")
    
    return result

### Search for Hashtag

In [29]:
def hashtag_search(hashtag_cache,start_time, end_time, string, Limit):
    
    at_begin_time = time.time_ns()
    
    query = start_time + end_time + string + 'LIMIT' + str(Limit)
    print(query)

    result = query_hashtag_cache(hashtag_cache,query,Limit)
#     print(result)
    if result == None:
        print("Going to tables")
        result = find_tweet_with_hashtag_in_time(start_time, end_time, Limit,string) 
#         print(result)
        if len(hashtag_cache) > 14 : 
            purge_key = find_oldest_lastused(hashtag_cache)
            hashtag_cache[purge_key]['pinned'] = ' '
            hashtag_cache = purge_cache(hashtag_cache)         
        
        if len(result) > 0:
            hashtag_cache = insert_hashtag_cache(hashtag_cache, query, result)
        else:
            result = "No results found"
    elif isinstance(result, list):
        print("found in Cache")
    else:
        print("purge")
        hashtag_cache[result]['pinned'] = ' '
        hashtag_cache = purge_cache(hashtag_cache)
        result = find_tweet_with_hashtag_in_time(start_time, end_time, Limit,string) 
        if len(result) > 0:
            hashtag_cache = insert_hashtag_cache(hashtag_cache, query, result)
        else:
#             print("No results found")
            result = "No results found in purge"
     
    at_end_time = time.time_ns()
    elapsed_time = at_end_time - at_begin_time
    print("Time taken:", elapsed_time, "seconds")    
        
    return result

In [30]:
!jupyter nbconvert --to script Cache_Load_Data.ipynb

[NbConvertApp] Converting notebook Cache_Load_Data.ipynb to script
[NbConvertApp] Writing 10527 bytes to Cache_Load_Data.py


In [31]:
#!jupyter nbconvert --to script --output Cache_Final_project_DB_User.py